# Introduction to Functional Programming in Haskell

## Haskell: What is it Good For?

![](https://imgs.xkcd.com/comics/haskell.png)

 * Haskell is very different from most other programming languages.
 * Haskell is used in industry (Facebook, Jane Street, Standard Chartered...).
 * Haskell has some cool advantages in the world of parallel processing.
 * Where Haskell really excels is in *building abstractions* (blessing and a curse).

## Equational Reasoning

 * Definitions in Haskell are actually that: definitions.
 * The rule is, if you have a definition **`LHS = RHS`**, then anywhere you see **`LHS`** in your code you can replace it with **`RHS`** and get the same result.
 * This means we cannot have multiple assignments of a variable in the same scope.

In [ ]:
x = 1
x

In [ ]:
x = 3
x = 4
x

## Types

 * Every expression in Haskell has a **type**.
 * We read the code "**`expr :: Type`**" as "expression **`expr`** has type **`Type`**".

In [ ]:
x = 3 :: Int

In [ ]:
:t x

In [ ]:
:t (x, x)

In [ ]:
:t [x, x, x, x]

In [ ]:
:t fst (x, x)

## Type Constraints

In [ ]:
:t 3

In [ ]:
:info Num

## Function Types (Currying)

In [ ]:
:t (+)

In [ ]:
:t (+ x)

In [ ]:
:t x + x

## Using Higher-Order Functions

In [ ]:
:t map

In [ ]:
:t map (+ x)

In [ ]:
map (+ 2) [1 .. 5]

In [ ]:
:t filter

In [ ]:
filter even [1..5]

In [ ]:
:t (.)

In [ ]:
:t filter (> 0) . map (+ x)

In [ ]:
:t foldl1

In [ ]:
foldl1 (+) [1..10]

## Lazy Evaluation

In a typical language, what happens when you evaluate something like this?

**```
x = (f(), g());
return x[0];```**

In [ ]:
:t error

In [ ]:
:t error "Fail!"

In [ ]:
error "Fail!"

In [ ]:
(1, error "Fail!")

In [ ]:
fst (1, error "Fail!")

All (well, most) values in Haskell are actually *promises* of a value.

`(1 + 2 + 3 :: Int)` is not stored as a binary number, but as a *computation that will result in a number*.

## Some Fun Stuff: Infinite Sequences

In [ ]:
list = map (+ 3) [1..] :: [Int]

In [ ]:
take 4 list

In [ ]:
fib = 1 : 1 : zipWith (+) fib (tail fib)

In [ ]:
take 10 fib

In [ ]:
take 5 (drop 50 fib)

## The Mathy Stuff: Functors

**Functors** are a concept taken from category theory. Think of them as anything that holds another value of arbitrary type.

```haskell
class Functor f where
    fmap :: (a -> b) -> f a -> f b
```

In [ ]:
:info Functor

In [ ]:
fmap (+2) [1,2,3,4]

In [ ]:
fmap (+2) ("hello", 2)

In [ ]:
:extension FlexibleContexts

import Diagrams.Prelude
import IHaskell.Display.Diagrams

node s = text s <> circle 0.7
box s = (translate (r2 (0.2, -0.2)) $
                   atop (translate (r2 (-0.8, 0.6)) $ text "F") $
                        node s)
        <> square 2.5

nodes = atPoints (trailVertices $ regPoly 4 6)
                 [ box "b" # named "bb", node "b" # named "nb"
                 , node "a" # named "na", box "a" # named "ba"]
        # connectOutside "na" "nb"
        # connectOutside "na" "ba"
        # connectOutside "ba" "bb"
        # connectOutside "nb" "bb"
        <> translate (r2 (-2.4, 0)) (text "F" # fontSize large # font "courier")
        <> translate (r2 ( 2.4, 0)) (text "F" # fontSize large # font "courier")
        <> translate (r2 (0, -2.4)) (text "fmap f" # fontSize large # font "courier")
        <> translate (r2 (0, 2.4)) (text "f" # fontSize large # font "courier")
        <> square 10 # dashingG [0.2, 0.05] 0

diagram nodes

But Functors are not just "containers"!

In [ ]:
:t fmap (> 50) (^ 2)

## Example: Writing our own List

In [ ]:
data List a = Cons a (List a)
            | Nil
    deriving (Show, Eq, Ord)

In [ ]:
y = Cons 1 (Cons 2 (Cons 3 Nil))
y

In [ ]:
length :: List a -> Int
length Nil = 0
length (Cons _ l) = 1 + length l

length y

In [ ]:
instance Functor List where
    fmap f Nil = Nil
    fmap f (Cons a l) = Cons (f a) (fmap f l)

In [ ]:
fmap (+3) y

## The Dreaded `Monad`

```haskell
class Functor m => Monad m where
    return :: a -> m a
    (>>=) :: m a -> (a -> m b) -> m b
    join :: m (m a) -> m a
```

In [ ]:
:t r2

In [ ]:
:t dashingL

In [ ]:
dReturn = centerX (atPoints [P $ r2 (0, 0), P $ r2 (5, 0)]
                            [node "a" # named "a", box "a" # named "b"])
          # connectOutside "a" "b"
          <> rect 9 3 # dashingG [0.2, 0.05] 0
          <> translate (r2 (-3.1, 1.2)) (text "return" # font "courier" # fontSize large)
          <> rect 10 3.5 # lc white

bbox s = (translate (r2 (0.4, -0.4)) $
                    atop (translate (r2 (-1.6, 1.3)) $ text "F") $
                         square 2.5
                         <> (atop (translate (r2 (-0.8, 0.6)) $ text "F") (node s)))
         <> square 3.5

dJoin = centerX (atPoints [P $ r2 (0, 0), P $ r2 (5, 0)] [bbox "a" # named "b", box "a" # named "a"])
        # connectOutside "b" "a"
        <> rect 9 7 # dashingG [0.2, 0.05] 0
        <> translate (r2 (-3.5, 3)) (text "join" # font "courier" # fontSize large)
        <> rect 10 7.5 # lc white

diagram $ dReturn === dJoin

## OK, But Why Monads?

Monads are just one abstraction we can apply to our code, but they end up being useful for a wide variety of things.

### The `Maybe` Monad

```haskell
data Maybe a = Just a
             | Nothing

instance Functor Maybe where
    fmap f (Just a) = Just (f a)
    fmap f Nothing = Nothing

instance Monad Maybe where
    return = Just
    
    (Just x) >>= f = Just (f x)
    Nothing >>= f = Nothing
```

In [ ]:
import Data.List (find)
:t find

In [ ]:
find even [1,3..9]

In [ ]:
find (> 5) [1..]

In [ ]:
l1 = [3,5..11]
l2 = [2,4..20]

find odd l1   >>=   \x -> find (> x^2) l2

### The `IO` Monad

Another place where monads have turned out to be very useful is in structuring IO operations.

**Note**: The following is totally wrong, but useful for understanding.
```haskell
type IO a = RealWorld -> (a, RealWorld)

instance Functor IO where
    fmap f io = \world -> let (x, world') = io world in (f x, world')

instance Monad IO where
    return x  =  \world -> (x, world)
    io1 >>= f  =  \world -> let (x', world') = io1 world in f x' world'
```

In [ ]:
:t putStrLn

In [ ]:
:t getLine

In [ ]:
:t getLine >>= putStrLn

In [ ]:
:t getLine >>= (\l1 -> getLine >>= (\l2 -> putStrLn (l1 ++ l2)))

This is going to get awkward really fast... Fortunately we have some syntactic sugar for this!

### `do` Notation, or Why Haskell is Also Imperative

In [ ]:
echo2 = do
    l1 <- getLine
    l2 <- getLine
    putStrLn (l1 ++ l2)

:t echo2

But it doesn't stop there! Because this notation only requires a monad to work, we can use it for any monad!

In [ ]:
maybeOddSqr l1 l2 = do
    a <- find odd l1
    find (> a^2) l2
    
maybeOddSqr [2, 4, 6, 5] [1, 3 .. 30]

We can even write code that doesn't care what monad we're using:

In [ ]:
mystery l1 l2 = do
    a <- l1
    b <- l2
    return (a + b)

:t mystery

In [ ]:
mystery (Just 3) (Just 4)

In [ ]:
mystery (fmap read getLine) (fmap read getLine)

In [ ]:
mystery [1..5] [2,4,6]

## Beyond Monads

 * **More Typeclasses**: there are a lot of these... ![](https://wiki.haskell.org/wikiupload/d/df/Typeclassopedia-diagram.png)
 * **Monad Transformers**: what if we want a structure that combines functionality from different monads?
 * **Software Transactional Memory**: a system for updating data structures in a fully transactional way. Haskell provides probably the most elegant implementation.
 * **Functional Data Structures**: to be efficient, every intermediate stage of a data structure needs to be used in the final product, unmodified. This results in some highly creative, interesting, and even useful designs.
 * **Exploiting Purity for Parallel Processing**: pretty simple in principle; because there are no side-effects, pure computations can be split off and run in parallel without any additional programmer work.
 * **Parsers, Applicative and Monadic**: applicatives and monads provide fitting abstractions for context-free and contextual grammars, respectively. These classes allow us to build efficient, well-typed parsers into our haskell programs.